<a href="https://colab.research.google.com/github/ryali93/UNMSM_programacion2019-1/blob/master/jupyter/clase12_spatial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install fiona
!pip install geopandas
!pip install descartes
!pip install mapclassify

## 01_Describe

In [0]:
import fiona
import pprint

In [0]:
data_path = "TM_WORLD_BORDERS-0.3.shp"
c = fiona.open(data_path)

In [0]:
print(c.driver)
print(c.crs)
print(len(c))
print(c.bounds)
pprint.pprint(c.schema)

## 02_Transform

In [0]:
import csv
from shapely.geometry import Point, mapping
from fiona import collection
from fiona.crs import from_epsg

In [0]:
schema = { 'geometry': 'Point', 'properties': { 'depth': 'float:7.3', 'magnitude' : 'float:7.3' } }

In [0]:
with collection("earthquakes.shp", "w", crs=from_epsg(4326), driver='ESRI Shapefile', schema=schema) as output:
    with open('earthquakes.csv', 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            point = Point(float(row['Longitude']), float(row['Latitude']))
            output.write({
                'properties': {
                    'depth': row['Depth'],'magnitude': row['Magnitude']
                },
                'geometry': mapping(point)
            })

## 03_Geocoding

In [0]:
from geopy.geocoders import Nominatim

In [0]:
geolocator = Nominatim()
location = geolocator.geocode("Estadio de la UNMSM, Avenida Carlos Germán Amezaga, Urbanizacion Las Brisas, Lima, CALLAO 02, Perú")

In [0]:
print(location.address)
print((location.latitude, location.longitude))
print(location.raw)

Geocoding inverso

In [0]:
geolocator = Nominatim()
location = geolocator.reverse("-12.057615, -77.082842")
print(location.address)

## 04_Geopandas

In [0]:
import geopandas as gp
%matplotlib inline
pth = "TM_WORLD_BORDERS-0.3.shp"
world = gp.GeoDataFrame.from_file(pth)

In [0]:
world.head(3)

In [0]:
world.plot()

In [0]:
world.plot(column='POP2005', scheme='QUANTILES', k=5, colormap='OrRd',  legend=True,  figsize=(16, 10))

In [0]:
world.plot(column='POP2005', scheme='fisher_jenks', k=5, colormap='OrRd',  legend=True, figsize=(16, 10))

***
## Ejercicio

Leer los datos

In [0]:
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import fiona

In [0]:
from datetime import datetime

In [0]:
data = pd.read_csv('earthquakes.csv')
data.head()

In [0]:
geometry = [Point(xy) for xy in zip(data.Longitude, data.Latitude)]
crs = {'init': 'epsg:4326'} #https://www.spatialreferen%matplotlib inline
geo_df.plot()ce.org/ref/epsg/4326/
geo_df = GeoDataFrame(data, crs=crs, geometry=geometry)

# geo_df.to_file(driver='ESRI Shapefile', filename='data.shp')

In [0]:
%matplotlib inline
geo_df.plot()

In [0]:
geo_df = geo_df.drop(columns=['DateTime'])
gjson = geo_df.to_crs(epsg='4326').to_json()

In [0]:
import folium

In [0]:
mapa = folium.Map([-8.783333, -74.866667], zoom_start=4, tiles='cartodbpositron')
points = folium.features.GeoJson(gjson)
mapa.add_children(points)
mapa